In [1]:
import os
from scipy.io import loadmat
from pathlib import Path
import h5py
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') # use a non-interactive backend such as Agg (for PNGs), PDF, SVG or PS.
import numpy as np
import pandas as pd
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
import mne
from mne.preprocessing import create_eog_epochs, compute_proj_eog

In [2]:
def mkraw_task(subject_id, tasktime, interpolate_bad_channels, highpass):
    # read the EEG data
    eeg_path = os.path.join('..', '..', '..', 'data', str(subject_id), str(subject_id) + '_' + str(tasktime) + '_withevent.mat')
    # eeg_data = loadmat(eeg_path)['eeg']
    # eeg_data[1:33] *= 1e-6 # convert to microvolts, including 31 eeg channels and 1 eog channel
    # Open the MATLAB v7.3 file using h5py
    with h5py.File(eeg_path, 'r') as file:
        eeg_data = np.array(file['eeg']).T  # Transpose might be needed based on data organization
        eeg_data[1:33] *= 1e-6  # convert to microvolts, including 31 eeg channels and 1 eog channel

    # get placement data of standard 10-20 system
    montage_1020 = mne.channels.make_standard_montage('standard_1020')
    positions_1020 = montage_1020._get_ch_pos()
    elec_coords_1020 = {ch_name: coord for ch_name, coord in positions_1020.items() if ch_name in montage_1020.ch_names}

    # Define channel names and types
    ch_names = ['Time'] + ['Fp1', 'Fp2', 
                        'AF3', 'AF4', 
                        'F7', 'F3', 'Fz', 'F4', 'F8',
                        'FC5', 'FC1', 'FC2', 'FC6',
                        'T7', 'C3', 'Cz', 'C4', 'T8',
                        'CP5', 'CP1', 'CP2', 'CP6',
                        'P7', 'P3', 'Pz', 'P4', 'P8',
                        'PO3', 'PO4',
                        'O1', 'O2'] + ['EOG'] + ['Trigger'] + ['rest', 'question', 'response']
    ch_types = ['misc'] + ['eeg'] * 31 + ['eog'] + ['misc'] + ['stim'] * 3

    # Create the info object
    info = mne.create_info(ch_names, sfreq=4800, ch_types=ch_types)
    # Create raw object
    raw = mne.io.RawArray(eeg_data, info)

    # manually add the placement of electrodes
    elec_coords = {
        'Fp1': elec_coords_1020['Fp1'],
        'Fp2': elec_coords_1020['Fp2'],
        'AF3': elec_coords_1020['AF3'],
        'AF4': elec_coords_1020['AF4'],
        'F7': elec_coords_1020['F7'],
        'F3': elec_coords_1020['F3'],
        'Fz': elec_coords_1020['Fz'],
        'F4': elec_coords_1020['F4'],
        'F8': elec_coords_1020['F8'],
        'FC5': elec_coords_1020['FC5'],
        'FC1': elec_coords_1020['FC1'],
        'FC2': elec_coords_1020['FC2'],
        'FC6': elec_coords_1020['FC6'],
        'T7': elec_coords_1020['T7'],
        'C3': elec_coords_1020['C3'],
        'Cz': elec_coords_1020['Cz'],
        'C4': elec_coords_1020['C4'],
        'T8': elec_coords_1020['T8'],
        'CP5': elec_coords_1020['CP5'],
        'CP1': elec_coords_1020['CP1'],
        'CP2': elec_coords_1020['CP2'],
        'CP6': elec_coords_1020['CP6'],
        'P7': elec_coords_1020['P7'],
        'P3': elec_coords_1020['P3'],
        'Pz': elec_coords_1020['Pz'],
        'P4': elec_coords_1020['P4'],
        'P8': elec_coords_1020['P8'],
        'PO3': elec_coords_1020['PO3'],
        'PO4': elec_coords_1020['PO4'],
        'O1': elec_coords_1020['O1'],
        'O2': elec_coords_1020['O2'],
    }

    # Create the montage object
    montage = mne.channels.make_dig_montage(elec_coords, coord_frame='head')

    # add info and montage to raw object
    raw.set_montage(montage)
    raw.info['subject_info'] = {'id': subject_id}
    raw.info['experimenter'] = 'pre-task' if tasktime == 1 else 'post-task'

    # preprocess the data
    # highpass filter
    raw.filter(highpass, None, picks=['eeg', 'eog'])
    raw.notch_filter(freqs=np.arange(100, 251, 50), picks=['eeg'], notch_widths=np.arange(100, 251, 50)/50)


    # mark bad channels
    if interpolate_bad_channels:
        bad_channels_df = pd.read_csv('bad_channels.csv')
        bad_channels_row = bad_channels_df[(bad_channels_df['subject id'] == subject_id) & (bad_channels_df['session'] == 'task') & (bad_channels_df['task time'] == ('pre' if tasktime == 1 else 'post'))]
        if not bad_channels_row.empty:
            bad_channels = bad_channels_row['bad channels'].values[0]
            if pd.notna(bad_channels):
                bad_channels_list = bad_channels.split(',')
                raw.info['bads'].extend(bad_channels_list)

    # handle EOG
    eog_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=True)
    eog_evoked = create_eog_epochs(raw, picks=eog_picks).average()
    eog_evoked.apply_baseline((None, None))
    eog_projs, _ = compute_proj_eog(raw, n_grad=0, n_mag=0, n_eeg=1, reject=None, no_proj=True)
    raw.add_proj(eog_projs).apply_proj()
    
    # re-reference
    raw.set_eeg_reference('average', projection=False)

    # interpolate
    if interpolate_bad_channels:
        # Interpolate bad channels
        if raw.info['bads']:
            raw.interpolate_bads(reset_bads=True) # remove the bads from info.
        print(bad_channels)
    
    # notch filter q=50, notch_widths = freq/q
    # raw.notch_filter(50, notch_widths=1)
    # raw.notch_filter(100, notch_widths=2)
    print('subject id: ' + str(subject_id) + ' tasktime: pre-task. DONE.' if tasktime == 1 else 'subject id: ' + str(subject_id) + ' tasktime: post-task. DONE.')
    raw_save_path = os.path.join('..', '..', '..', 'data', str(subject_id), 'raw_' + str(subject_id) + '_' + str(tasktime) + '.fif')
    raw.save(raw_save_path, overwrite=True)

    # fig = mne.viz.plot_raw_psd(raw, fmax=150, spatial_colors=True)
    # plot_save_path = os.path.join('..', '..', '..', 'data', 'psd', str(subject_id) + '_' + before_or_after + '.png')
    # fig.savefig(plot_save_path)

In [3]:
# main
# script_dir = os.path.dirname(os.path.realpath(__file__))
# os.chdir(script_dir)
for subject_id in range (1, 27):
    for tasktime in range(1, 3):
        mkraw_task(subject_id, tasktime, interpolate_bad_channels=True, highpass=0.3)

Creating RawArray with float64 data, n_channels=37, n_times=3865076
    Range : 0 ... 3865075 =      0.000 ...   805.224 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Filter length: 52801 samples (11.000 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.6s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 31681 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.7s


Using EOG channel: EOG
EOG channel index for this subject is: [32]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 48000 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 79 significant peaks
Number of EOG events detected: 79
Not setting metadata
79 matching events found
No baseline correction applied
Using data from preloaded Raw for 79 events and 4801 original time points ...
0 bad epochs dropped
Applying baseline correction (m

...

Using EOG channel: EOG
EOG channel index for this subject is: [32]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 48000 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 571 significant peaks
Number of EOG events detected: 571
Not setting metadata
571 matching events found
No baseline correction applied
Using data from preloaded Raw for 571 events and 4801 original time points ...
1 bad epochs dropped
Applying baseline correctio

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.8s


Not setting metadata
571 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 571 events and 1921 original time points ...
0 bad epochs dropped
No channels 'grad' found. Skipping.
No channels 'mag' found. Skipping.
Adding projection: eeg--0.200-0.200-PCA-01
Done.
1 projection items deactivated
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Created an SSP operator (subspace dimension = 1)
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 92.8 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 1 sensors
O2
subject id: 26 tasktime: post-task. DONE.
Overwriting existing file.
Writing c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\int

C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_19292\662339870.py:114: RuntimeWarning: This filename (c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\intelligence_tRNS\tRNS-Intelligence\processing\prepare_raw_file\..\..\..\data\26\raw_26_2.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(raw_save_path, overwrite=True)


Closing c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\intelligence_tRNS\tRNS-Intelligence\processing\prepare_raw_file\..\..\..\data\26\raw_26_2.fif
[done]


In [4]:
def mkraw_rest(subject_id, tasktime, interpolate_bad_channels, highpass):
    # read the EEG data
    eeg_path = os.path.join('..', '..', '..', 'data', str(subject_id), str(subject_id) + '_rs' + str(tasktime) + '_withevent.mat')
    # eeg_data = loadmat(eeg_path)['eeg']
    # eeg_data[1:33] *= 1e-6 # convert to microvolts, including 31 eeg channels and 1 eog channel
    # Open the MATLAB v7.3 file using h5py
    with h5py.File(eeg_path, 'r') as file:
        eeg_data = np.array(file['eeg']).T  # Transpose might be needed based on data organization
        eeg_data[1:33] *= 1e-6  # convert to microvolts, including 31 eeg channels and 1 eog channel

    # get placement data of standard 10-20 system
    montage_1020 = mne.channels.make_standard_montage('standard_1020')
    positions_1020 = montage_1020._get_ch_pos()
    elec_coords_1020 = {ch_name: coord for ch_name, coord in positions_1020.items() if ch_name in montage_1020.ch_names}

    # Define channel names and types
    ch_names = ['Time'] + ['Fp1', 'Fp2', 
                        'AF3', 'AF4', 
                        'F7', 'F3', 'Fz', 'F4', 'F8',
                        'FC5', 'FC1', 'FC2', 'FC6',
                        'T7', 'C3', 'Cz', 'C4', 'T8',
                        'CP5', 'CP1', 'CP2', 'CP6',
                        'P7', 'P3', 'Pz', 'P4', 'P8',
                        'PO3', 'PO4',
                        'O1', 'O2'] + ['EOG'] + ['Trigger'] + ['rest']
    ch_types = ['misc'] + ['eeg'] * 31 + ['eog'] + ['misc'] + ['stim']

    # Create the info object
    info = mne.create_info(ch_names, sfreq=4800, ch_types=ch_types)
    # Create raw object
    raw = mne.io.RawArray(eeg_data, info)

    # manually add the placement of electrodes
    elec_coords = {
        'Fp1': elec_coords_1020['Fp1'],
        'Fp2': elec_coords_1020['Fp2'],
        'AF3': elec_coords_1020['AF3'],
        'AF4': elec_coords_1020['AF4'],
        'F7': elec_coords_1020['F7'],
        'F3': elec_coords_1020['F3'],
        'Fz': elec_coords_1020['Fz'],
        'F4': elec_coords_1020['F4'],
        'F8': elec_coords_1020['F8'],
        'FC5': elec_coords_1020['FC5'],
        'FC1': elec_coords_1020['FC1'],
        'FC2': elec_coords_1020['FC2'],
        'FC6': elec_coords_1020['FC6'],
        'T7': elec_coords_1020['T7'],
        'C3': elec_coords_1020['C3'],
        'Cz': elec_coords_1020['Cz'],
        'C4': elec_coords_1020['C4'],
        'T8': elec_coords_1020['T8'],
        'CP5': elec_coords_1020['CP5'],
        'CP1': elec_coords_1020['CP1'],
        'CP2': elec_coords_1020['CP2'],
        'CP6': elec_coords_1020['CP6'],
        'P7': elec_coords_1020['P7'],
        'P3': elec_coords_1020['P3'],
        'Pz': elec_coords_1020['Pz'],
        'P4': elec_coords_1020['P4'],
        'P8': elec_coords_1020['P8'],
        'PO3': elec_coords_1020['PO3'],
        'PO4': elec_coords_1020['PO4'],
        'O1': elec_coords_1020['O1'],
        'O2': elec_coords_1020['O2'],
    }

    # Create the montage object
    montage = mne.channels.make_dig_montage(elec_coords, coord_frame='head')

    # add info and montage to raw object
    raw.set_montage(montage)
    raw.info['subject_info'] = {'id': subject_id}
    raw.info['experimenter'] = 'pre-task' if tasktime == 1 else 'post-task'
    # preprocess the data
    # highpass filter
    raw.filter(highpass, None, picks=['eeg', 'eog'])
    # notch filter q=50, notch_widths = freq/q
    raw.notch_filter(freqs=np.arange(100, 251, 50), picks=['eeg'], notch_widths=np.arange(100, 251, 50)/50)

    # mark bad channels
    if interpolate_bad_channels:
        bad_channels_df = pd.read_csv('bad_channels.csv')
        bad_channels_row = bad_channels_df[(bad_channels_df['subject id'] == subject_id) & (bad_channels_df['session'] == 'rest') & (bad_channels_df['task time'] == ('pre' if tasktime == 1 else 'post'))]
        if not bad_channels_row.empty:
            bad_channels = bad_channels_row['bad channels'].values[0]
            if pd.notna(bad_channels):
                bad_channels_list = bad_channels.split(',')
                raw.info['bads'].extend(bad_channels_list)

    # handle EOG
    eog_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=True)
    eog_evoked = create_eog_epochs(raw, picks=eog_picks).average()
    eog_evoked.apply_baseline((None, None))
    eog_projs, _ = compute_proj_eog(raw, n_grad=0, n_mag=0, n_eeg=1, reject=None, no_proj=True)
    raw.add_proj(eog_projs).apply_proj()
    
    # re-reference
    raw.set_eeg_reference('average', projection=False)

    # interpolate
    if interpolate_bad_channels:
        # Interpolate bad channels
        if raw.info['bads']:
            raw.interpolate_bads(reset_bads=True) # remove the bads from info.
        print(bad_channels)
    

    print('subject id: ' + str(subject_id) + ' tasktime: pre-rest. DONE.' if tasktime == 1 else 'subject id: ' + str(subject_id) + ' tasktime: post-rest. DONE.')
    raw_save_path = os.path.join('..', '..', '..', 'data', str(subject_id), 'raw_' + str(subject_id) + '_rs' + str(tasktime) + '.fif')
    raw.save(raw_save_path, overwrite=True)

    # fig = mne.viz.plot_raw_psd(raw, fmax=150, spatial_colors=True)
    # plot_save_path = os.path.join('..', '..', '..', 'data', 'psd', str(subject_id) + '_' + before_or_after + '.png')
    # fig.savefig(plot_save_path)

In [5]:
for subject_id in range (1, 27):
    for tasktime in range(1, 3):
        mkraw_rest(subject_id, tasktime, interpolate_bad_channels=True, highpass=0.3)

Creating RawArray with float64 data, n_channels=35, n_times=864131
    Range : 0 ... 864130 =      0.000 ...   180.027 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Filter length: 52801 samples (11.000 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 31681 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Using EOG channel: EOG
EOG channel index for this subject is: [32]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 48000 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 10 significant peaks
Number of EOG events detected: 10
Not setting metadata
10 matching events found
No baseline correction applied
Using data from preloaded Raw for 10 events and 4801 original time points ...
0 bad epochs dropped
Applying baseline correction (m

...

Using EOG channel: EOG
EOG channel index for this subject is: [32]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 48000 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 10 significant peaks
Number of EOG events detected: 10
Not setting metadata
10 matching events found
No baseline correction applied
Using data from preloaded Raw for 10 events and 4801 original time points ...
0 bad epochs dropped
Applying baseline correction (m

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.9s


Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 1921 original time points ...
0 bad epochs dropped
No channels 'grad' found. Skipping.
No channels 'mag' found. Skipping.
Adding projection: eeg--0.200-0.200-PCA-01
Done.
1 projection items deactivated
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Created an SSP operator (subspace dimension = 1)
nan
subject id: 26 tasktime: post-rest. DONE.
Overwriting existing file.
Writing c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\intelligence_tRNS\tRNS-Intelligence\processing\prepare_raw_file\..\..\..\data\26\raw_26_rs2.fif


C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_19292\476344993.py:111: RuntimeWarning: This filename (c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\intelligence_tRNS\tRNS-Intelligence\processing\prepare_raw_file\..\..\..\data\26\raw_26_rs2.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(raw_save_path, overwrite=True)


Closing c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\intelligence_tRNS\tRNS-Intelligence\processing\prepare_raw_file\..\..\..\data\26\raw_26_rs2.fif
[done]
